In [2]:
import calliope
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
hydrogen_cost = np.arrange(0,0.25,0.01) #range of hydrogen costs, to test the cost competitiveness of Green hydrogen production method

In [ ]:
def cost_iteration(scenario,)